In [1]:
%load_ext autoreload
%autoreload 2
import torch

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load model/data

In [2]:
from datasets.audioset import Audioset
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets.EmergencyDataset import EmergencyDataset

audio_set_valA = Audioset(split_mode='validation', fixed_padding=True)
#audio_set_valB = Audioset(split_mode='validation', fixed_padding=False)
#emergency_set_val = EmergencyDataset('val')
# val_loader = DataLoader(audio_set_val, batch_size=2, collate_fn=audio_set_val.pad_seq)
# valid_loader = DataLoader(audio_set_valA, batch_size=1)
# valid_loader = DataLoader(audio_set_valA, batch_size=1)

In [29]:
from classification.models.crnn.model import AudioCRNN
# val_loader = DataLoader(audio_set_val, batch_size=2, collate_fn=audio_set_val.pad_seq)

res_path = '/nfs/students/summer-term-2020/project-4/data/models/crnn.pth'
checkpoint = torch.load(res_path)

model = AudioCRNN(checkpoint['classes'], checkpoint['config'])
model.load_state_dict(checkpoint['state_dict'])
model = model.eval().cuda()

In [13]:
from classification.models.SpectrogramCNN import SpectrogramCNN

state_dict_path = '/nfs/students/summer-term-2020/project-4/data/models/SpectrogramBasedCNN.pt'

model = SpectrogramCNN()
model.load_state_dict(torch.load(state_dict_path))
model = model.cuda()
model = model.eval()

In [3]:
from classification.models.DeepRecursiveCNN import DeepRecursiveCNN

model_state_dict_path = "/nfs/students/summer-term-2020/project-4/yan/models/best_model_state_dict.pt"
model = DeepRecursiveCNN()
model.load_state_dict(torch.load(model_state_dict_path))
model = model.eval()

In [40]:
from classification.models.M5 import M5

loaded_dict = torch.load("/nfs/homedirs/herrmanp/project-4/experiments/notebooks/pascal/adv_totaly_hig_50epochs.pt")
model = M5(loaded_dict["hparams"])
model.load_state_dict(loaded_dict["state_dict"])
model = model.eval().cuda()

## FGSM

In [71]:
from attacks.fga import FastGradientAttack

valid_loader = DataLoader(audio_set_valA, batch_size=10)
#val_loader = DataLoader(audio_set_valB, batch_size=1, collate_fn=audio_set_val.pad_seq)
#valid_loader = DataLoader(emergency_set_val, batch_size=1)

attack_parameters = {'epsilon': 0.001}

fgsm = FastGradientAttack(model, valid_loader, attack_parameters)
fgsm.attack()

100%|██████████| 169/169 [00:13<00:00, 12.64it/s]


In [74]:
# fgsm.showAdversarialExample(target_class=1)

In [75]:
fgsm.report()

Attack-Parameters:	{'epsilon': 0.001}
Early stopping: 	False (-1)

Successfully attacked:	20
Total attacked: 	1483
Total processed:	1687

Success-Rate: 		0.01
Perturbed Accurracy: 	0.87


## PGD

In [34]:
from attacks.pgd import PGD

#valid_loader = DataLoader(audio_set_val, batch_size=1)
valid_loader = DataLoader(emergency_set_val, batch_size=1)
attack_parameters = {'epsilon': 0.5, 'num_iter': 30}

pgd = PGD(model, valid_loader, attack_parameters, early_stopping=20)
pgd.attack()

 20%|██        | 343/1687 [00:26<01:44, 12.89it/s]

Early stopping


In [ ]:
# pgd.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [58]:
pgd.report()

Attack-Parameters:	{'epsilon': 0.5, 'num_iter': 30}
Early stopping: 	True (20)

Successfully attacked:	20
Total attacked: 	40
Total processed:	44

Success-Rate: 		0.5
Perturbed Accurracy: 	0.45


## Volume attacks

In [ ]:
from attacks.volume import VolumeAttack

valid_loader = DataLoader(audio_set_valA, batch_size=1)
#valid_loader = DataLoader(emergency_set_val, batch_size=1)
attack_parameters = {'epsilon': 0.1, 'num_iter': 10}

vol = VolumeAttack(model, valid_loader, attack_parameters, early_stopping=3)
vol.attack()

In [112]:
# vol.showAdversarialExample(target_class=1)

In [36]:
vol.report()

Attack-Parameters:	{'epsilon': 0.1, 'num_iter': 10}
Early stopping: 	True (3)

Successfully attacked:	0
Total attacked: 	1356
Total processed:	1687

Success-Rate: 		0.0
Perturbed Accurracy: 	0.8


## Interpolation attack

In [ ]:
import librosa
path_tum_sound = '/nfs/students/summer-term-2020/project-4/yan/tum.wav'
tum_sound,sr = librosa.load(path_tum_sound, sr=48000)

max_length = len(audio_set_valA[0][0])
padding = int((max_length - len(tum_sound))/2)
zero_padded_data = torch.zeros(max_length)
zero_padded_data[padding:padding+tum_sound.shape[0]] = torch.from_numpy(tum_sound)
tum_sound = zero_padded_data

In [ ]:
from attacks.interpolation import InterpolationAttack

valid_loader = DataLoader(audio_set_val, batch_size=1)
attack_parameters = {'epsilon': 0.2, 'num_iter': 30, 'overlay_sound': tum_sound}

ia = InterpolationAttack(model, valid_loader, attack_parameters, early_stopping=1)
ia.attack()

In [298]:
# ia.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [297]:
ia.report()

Attack-Parameters:	{'epsilon': 0.2, 'num_iter': 30, 'overlay_sound': tensor([0., 0., 0.,  ..., 0., 0., 0.])}
Early stopping: 	True (1)

Successfully attacked:	1
Total attacked: 	67
Total processed:	75

Success-Rate: 		0.01
Perturbed Accurracy: 	0.88


## Time stretching attack

In [ ]:
from attacks.speed import TimeStretchAttack

valid_loader = DataLoader(audio_set_val, batch_size=1)
attack_parameters = {'num_iter': 30, 'lower': 0.1, 'upper': 0.5}

tsa = TimeStretchAttack(model, valid_loader, attack_parameters, early_stopping=5)
tsa.attack()

In [ ]:
tsa.showAdversarialExample(target_class=0)

In [170]:
tsa.report()

Attack-Parameters:	{'num_iter': 30, 'lower': 0.1, 'upper': 1}
Early stopping: 	True (20)

Successfully attacked:	16
Total attacked: 	1483
Total processed:	1687

Success-Rate: 		0.01
Perturbed Accurracy: 	0.87


## Pitch attack

In [ ]:
from attacks.pitch import PitchAttack

valid_loader = DataLoader(audio_set_val, batch_size=1)
attack_parameters = {'num_iter': 5, 'lower': 0, 'upper': 10}

pa = PitchAttack(model, valid_loader, attack_parameters, early_stopping=10)
pa.attack()

In [6]:
#pa.showAdversarialExample(target_class=0)

In [9]:
pa.report()

Attack-Parameters:	{'num_iter': 5, 'lower': 0, 'upper': 10}
Early stopping: 	True (10)

Successfully attacked:	10
Total attacked: 	87
Total processed:	98

Success-Rate: 		0.11
Perturbed Accurracy: 	0.79
